In [1]:
import pandas as pd
import numpy as np

file = "Resources/purchase_data.csv"

game_data = pd.read_csv(file)
game_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
#player count
players = len(game_data["SN"].value_counts())
player_count = pd.DataFrame([players], columns = ["Total Players"])
player_count

,Total Players
0,576


In [3]:
#Purchasing Analysis (Total)
Unique_items = len(game_data["Item Name"].unique())
Avg_price = game_data["Price"].mean()
Number_Purchases = len(game_data["Item Name"])
Revenue = game_data["Price"].sum()

Purchasing_Analysis_Total = pd.DataFrame({"Number of Unique Items": [Unique_items],
                                           "Average Price": [Avg_price],
                                           "Number of Purchases": [Number_Purchases],
                                           "Total Revenue": [Revenue]})

Purchasing_Analysis_Total["Average Price"] = Purchasing_Analysis_Total["Average Price"].map("${:.2f}".format)
Purchasing_Analysis_Total["Total Revenue"] = Purchasing_Analysis_Total["Total Revenue"].map("${:.2f}".format)
Purchasing_Analysis_Total = Purchasing_Analysis_Total[["Number of Unique Items", "Average Price", "Number of Purchases", "Total Revenue"]]
Purchasing_Analysis_Total

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [4]:
#Gender Demographics
gender = game_data[["SN", "Gender"]]
gender = gender.drop_duplicates()
counts = gender["Gender"].value_counts()

total_counts = [counts[0],counts[1],counts[2]]
percents = [round((counts[0]/players)*100,2),round((counts[1]/players)*100,2),round((counts[2]/players)*100,2)]


gender_demo = pd.DataFrame({ "Percentage of Players": percents,"Total Count": total_counts})
gender_demo.index = (["Male", "Female", "Other / Non-Disclosed"])
gender_demo

,Percentage of Players,Total Count
Male,84.03,484
Female,14.06,81
Other / Non-Disclosed,1.91,11


In [5]:
#Purchasing Analysis (Gender)
gender = game_data.groupby(["Gender"])

purch_Count = gender["SN"].count()
purch_Price = gender["Price"].mean()
purch_Value = gender["Price"].sum()

duplicates = game_data.drop_duplicates(subset='SN', keep="first")
grouped_dup = duplicates.groupby(["Gender"])

purch_Norm = (gender["Price"].sum() / grouped_dup["SN"].count())

Purch_Anal_Gen = pd.DataFrame({"Purchase Count": purch_Count,
                              "Average Purchase Price": purch_Price,
                              "Total Purchase Value": purch_Value,
                              "Normalized Totals": purch_Norm})

Purch_Anal_Gen["Average Purchase Price"] = Purch_Anal_Gen["Average Purchase Price"].map("${:.2f}".format)
Purch_Anal_Gen["Total Purchase Value"] = Purch_Anal_Gen["Total Purchase Value"].map("${:.2f}".format)
Purch_Anal_Gen["Normalized Totals"] = Purch_Anal_Gen["Normalized Totals"].map("${:.2f}".format)
Purch_Anal_Gen = Purch_Anal_Gen[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]
Purch_Anal_Gen

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [6]:
#Age Demographics
age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
groups = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

game_data["Age Demographics"]= pd.cut(game_data["Age"], age_bins, labels = groups)
demographic_group = game_data.groupby("Age Demographics")

game_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Demographics
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24


In [7]:
#Top Spenders
top_spenders = game_data['Item ID'].groupby(game_data['SN']).count()

top_spenders= pd.DataFrame(data=top_spenders)
top_spenders.columns = ['Purchase Count']

top_spenders['Average Purchase Price'] = round(game_data['Price'].groupby(game_data['SN']).mean(),2)
top_spenders['Total Purchase Value'] = game_data['Price'].groupby(game_data['SN']).sum()

top_spenders.sort_values(by=['Total Purchase Value'], ascending=False, inplace=True)

top_spenders['Average Purchase Price'] = top_spenders['Average Purchase Price'].map('${:,.2f}'.format)
top_spenders['Total Purchase Value'] = top_spenders['Total Purchase Value'].map('${:,.2f}'.format)
top_spenders.head()


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [8]:
#Most Popular Items
most_pop = game_data.groupby(['Item ID', 'Item Name','Price'])['Price'].agg(['count','sum'])
most_pop.columns = ['Purchase Count', 'Total Purchase Value']

most_pop.reset_index(inplace=True)
most_pop.set_index(['Item ID','Item Name'] ,inplace=True)
most_pop = most_pop[['Purchase Count', 'Price', 'Total Purchase Value']]

most_popular_items = most_pop.sort_values(by='Purchase Count', ascending=False)

most_popular_items['Price'] = most_popular_items['Price'].map('${:,.2f}'.format)
most_popular_items['Total Purchase Value'] = most_popular_items['Total Purchase Value'].map('${:,.2f}'.format)
most_popular_items.head()

,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [9]:
#Most Profitable Items
most_profitable_items = most_pop.sort_values(by='Total Purchase Value', ascending=False)

most_profitable_items['Price'] = most_profitable_items['Price'].map('${:,.2f}'.format)
most_profitable_items['Total Purchase Value'] = most_profitable_items['Total Purchase Value'].map('${:,.2f}'.format)
most_profitable_items.head()

,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
